In [3]:
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
import itertools
from itertools import permutations
from itertools import combinations
from ROOT import *
import uproot3
import awkward as ak
import vector

In [4]:
arrays1 = (uproot3.open("/Users/arleeshelby/BdToJpsi_prompt_PU140.root")["L1TrackNtuple/eventTree"].arrays("*", namedecode="utf-8"))

In [5]:
arrays1.keys()

dict_keys(['trk_pt', 'trk_eta', 'trk_phi', 'trk_d0', 'trk_z0', 'trk_chi2', 'trk_chi2rphi', 'trk_chi2rz', 'trk_bendchi2', 'trk_nstub', 'trk_lhits', 'trk_dhits', 'trk_seed', 'trk_hitpattern', 'trk_phiSector', 'trk_genuine', 'trk_loose', 'trk_unknown', 'trk_combinatoric', 'trk_fake', 'trk_matchtp_pdgid', 'trk_matchtp_pt', 'trk_matchtp_eta', 'trk_matchtp_phi', 'trk_matchtp_z0', 'trk_matchtp_dxy', 'trk_parent_matchtp_pdgid', 'trk_parent_matchtp_m', 'trk_parent_matchtp_m_calc', 'trk_parent_matchtp_charge', 'trk_parent_matchtp_vz', 'tp_pt', 'tp_eta', 'tp_phi', 'tp_dxy', 'tp_d0', 'tp_z0', 'tp_d0_prod', 'tp_z0_prod', 'tp_pdgid', 'tp_nmatch', 'tp_nstub', 'tp_eventid', 'tp_charge', 'parent_tp_pdgid', 'parent_tp_m', 'parent_tp_m_calc', 'parent_tp_charge', 'parent_tp_vz', 'matchtrk_pt', 'matchtrk_eta', 'matchtrk_phi', 'matchtrk_z0', 'matchtrk_d0', 'matchtrk_chi2', 'matchtrk_chi2rphi', 'matchtrk_chi2rz', 'matchtrk_bendchi2', 'matchtrk_nstub', 'matchtrk_lhits', 'matchtrk_dhits', 'matchtrk_seed', 'mat

In [6]:
parent_ID = arrays1['parent_tp_pdgid']
daughter_ID = arrays1['tp_pdgid']
daughter_pt = arrays1['tp_pt']
daughter_phi = arrays1['tp_phi']
daughter_eta = arrays1['tp_eta']
tp_eventid = arrays1['tp_eventid']
tp_charge = arrays1['tp_charge']
pt_match = arrays1['matchtrk_pt']

In [7]:
other_parameters = []
for i in range(len(arrays1['matchtrk_z0'])):
    other_parameters.append(np.dstack((arrays1['matchtrk_z0'][i],arrays1['matchtrk_d0'][i]))[0])

In [8]:
other_parameters_reco = []
for i in range(len(arrays1['trk_z0'])):
    other_parameters_reco.append(np.dstack((arrays1['trk_z0'][i],arrays1['trk_d0'][i]))[0])

In [9]:
len(other_parameters_reco[0])

134

In [10]:
other_parameters_tp = []
for i in range(len(arrays1['tp_z0'])):
    other_parameters_tp.append(np.dstack((arrays1['tp_z0'][i],arrays1['tp_d0'][i]))[0])

In [11]:
mass_mu = 0.1056583755

In [12]:
mass_array = []
for i in range(len(arrays1['matchtrk_pt'])):
    mass_array.append([mass_mu]*len(arrays1['matchtrk_pt'][i]))

In [13]:
mass_array_reco = [mass_mu]*len(arrays1['trk_pt'])

In [14]:
len(mass_array_reco)

9389

In [15]:
momentum_vector = vector.zip({'pt':arrays1['matchtrk_pt'], 'eta':arrays1['matchtrk_eta'], 'phi':arrays1['matchtrk_phi'],'mass':mass_array})

In [16]:
momentum_vector_tp = vector.zip({'pt':arrays1['tp_pt'], 'eta':arrays1['tp_eta'], 'phi':arrays1['tp_phi'],'mass':mass_array})

In [17]:
momentum_vector_reco = vector.zip({'pt':arrays1['trk_pt'], 'eta':arrays1['trk_eta'], 'phi':arrays1['trk_phi'],'mass':mass_array_reco})

In [18]:
momentum_vector_reco[0]

<MomentumArray4D [{rho: 2.46, phi: -0.431, ...}, ...] type='134 * Momentum4...'>

In [19]:
len(momentum_vector_tp[0])

132

In [20]:
match_mask = []
for i in range(len(pt_match)):
    match_mask.append([pt_match[i][j]>0 for j in range(len(pt_match[i]))])

In [21]:
len(match_mask[0])

132

In [22]:
daughter_ID_matched = [daughter_ID[i][match_mask[i]] for i in range(len(daughter_ID))]

In [23]:
len(daughter_ID_matched[0])

117

In [24]:
matched_tracks = [momentum_vector[i][match_mask[i]] for i in range(len(momentum_vector))]

In [25]:
matched_tracks_tp = [momentum_vector_tp[i][match_mask[i]] for i in range(len(momentum_vector_tp))]

In [26]:
matched_parameters = [other_parameters[i][match_mask[i]] for i in range(len(other_parameters))]

In [27]:
matched_parameters_tp = [other_parameters_tp[i][match_mask[i]] for i in range(len(other_parameters_tp))]

In [28]:
len(other_parameters)

9389

In [29]:
muon_mask_pos = []
for i in range(len(daughter_ID_matched)):
    muon_mask_pos.append([daughter_ID_matched[i][j]==13 for j in range(len(daughter_ID_matched[i]))])
    
muon_mask_neg = []
for i in range(len(daughter_ID_matched)):
    muon_mask_neg.append([daughter_ID_matched[i][j]==-13 for j in range(len(daughter_ID_matched[i]))])

In [30]:
muon_mask_trk_match = []
for i in range(len(arrays1['trk_matchtp_pdgid'])):
    muon_mask_trk_match.append([np.abs(arrays1['trk_matchtp_pdgid'][i][j])==13 for j in range(len(arrays1['trk_matchtp_pdgid'][i]))])

In [31]:
a = [np.where(sum(muon_mask_trk_match[i])>1) for i in range(len(muon_mask_trk_match))]

In [32]:
sum(muon_mask_trk_match[5])

3

In [33]:
muon_momentum_vector_pos = [matched_tracks[i][muon_mask_pos[i]] for i in range(len(matched_tracks))]

muon_momentum_vector_neg = [matched_tracks[i][muon_mask_neg[i]] for i in range(len(matched_tracks))]

In [34]:
muon_momentum_vector_pos_tp = [matched_tracks_tp[i][muon_mask_pos[i]] for i in range(len(matched_tracks_tp))]

muon_momentum_vector_neg_tp = [matched_tracks_tp[i][muon_mask_neg[i]] for i in range(len(matched_tracks_tp))]

In [35]:
len(muon_momentum_vector_pos[0])

1

In [36]:
other_parameters_pos = [matched_parameters[i][muon_mask_pos[i]] for i in range(len(matched_parameters))]
other_parameters_neg = [matched_parameters[i][muon_mask_neg[i]] for i in range(len(matched_parameters))]

In [37]:
other_parameters_pos_tp = [matched_parameters_tp[i][muon_mask_pos[i]] for i in range(len(matched_parameters_tp))]
other_parameters_neg_tp = [matched_parameters_tp[i][muon_mask_neg[i]] for i in range(len(matched_parameters_tp))]

In [38]:
len(matched_parameters_tp)

9389

In [39]:
new_positive_mu = []
new_negative_mu = []
for i in range(9389):
    posParticles = muon_momentum_vector_pos[i]
    negParticles = muon_momentum_vector_neg[i]

    lengthOfNeg = len(negParticles)
    lengthOfPos = len(posParticles)
    
    newPosArray = []
    newNegArray = []
    
    if lengthOfNeg==0 or lengthOfPos==0:
        continue;

    
    for i in range(len(posParticles)):
        newPosArray.append([[posParticles[i]]*lengthOfNeg][0])

    for j in range(lengthOfPos):
        for i in negParticles:
            newNegArray.append(i)

    new_positive_mu.append(newPosArray)
    new_negative_mu.append(newNegArray)

In [40]:
new_positive_mu[0]

[[<MomentumRecord4D {rho: 4.14, phi: 0.0673, eta: ..., ...} type='Momentum4D[...'>,
  <MomentumRecord4D {rho: 4.14, phi: 0.0673, eta: ..., ...} type='Momentum4D[...'>]]

In [41]:
new_positive_mu_tp = []
new_negative_mu_tp = []
for i in range(9389):
    posParticles = muon_momentum_vector_pos_tp[i]
    negParticles = muon_momentum_vector_neg_tp[i]

    lengthOfNeg = len(negParticles)
    lengthOfPos = len(posParticles)
    
    newPosArray = []
    newNegArray = []

    if lengthOfNeg==0 or lengthOfPos==0:
        continue;

    
    for i in range(len(posParticles)):
        newPosArray.append([[posParticles[i]]*lengthOfNeg][0])

    for j in range(lengthOfPos):
        for i in negParticles:
            newNegArray.append(i)
    
    new_positive_mu_tp.append(newPosArray)
    new_negative_mu_tp.append(newNegArray)

In [42]:
#delta d0 and z0 array matchtrk
pos_mu_prams = []
neg_mu_prams = []
for i in range(9389):
    posParticles = other_parameters_pos[i]
    negParticles = other_parameters_neg[i]

    lengthOfNeg = len(negParticles)
    lengthOfPos = len(posParticles)
    
    newPosArray = []
    newNegArray = []

    if lengthOfNeg==0 or lengthOfPos==0:
        continue;

    
    for i in range(len(posParticles)):
        newPosArray.append([[posParticles[i]]*lengthOfNeg][0])

    for j in range(lengthOfPos):
        for i in negParticles:
            newNegArray.append(i)  
    
    pos_mu_prams.append(newPosArray)
    neg_mu_prams.append(newNegArray)

In [43]:
#delta d0 and z0 array tp
pos_mu_prams_tp = []
neg_mu_prams_tp = []
for i in range(9389):
    posParticles = other_parameters_pos_tp[i]
    negParticles = other_parameters_neg_tp[i]

    lengthOfNeg = len(negParticles)
    lengthOfPos = len(posParticles)
    
    newPosArray = []
    newNegArray = []

    if lengthOfNeg==0 or lengthOfPos==0:
        continue;

    
    for i in range(len(posParticles)):
        newPosArray.append([[posParticles[i]]*lengthOfNeg][0])

    for j in range(lengthOfPos):
        for i in negParticles:
            newNegArray.append(i)  
    
    pos_mu_prams_tp.append(newPosArray)
    neg_mu_prams_tp.append(newNegArray)

In [44]:
len(new_positive_mu_tp)

1554

In [45]:
# momentum_vector_1st_half = momentum_vector_reco[:len(momentum_vector_reco)//2]
# momentum_vector_2nd_half = momentum_vector_reco[len(momentum_vector_reco)//2:]

In [46]:
# other_parameters_1st_half = other_parameters_reco[:len(other_parameters_reco)//2]
# other_parameters_2nd_half = other_parameters_reco[len(other_parameters_reco)//2:]

In [47]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [48]:
reco_event_1_pos = []
reco_event_1_neg = []
for i in range(9389):
    posParticles = momentum_vector_reco[i]
    negParticles = momentum_vector_reco[i]

    lengthOfNeg = len(negParticles)
    lengthOfPos = len(posParticles)
    
    newPosArray = []
    newNegArray = []
    
    if lengthOfNeg==0 or lengthOfPos==0:
        continue;

    for l in range(len(posParticles)):
        reco_event_1_pos.append([[posParticles[l]]*(lengthOfNeg-l)][0])
    
    for j in range(len(posParticles)):
        newNegArray.append(negParticles[j:])
        
    a = flatten(newNegArray)
    reco_event_1_neg.append(a)

In [49]:
reco_event_1_pos_parameters = []
reco_event_1_neg_parameters = []
for i in range(9389):
    posParticles = other_parameters_reco[i]
    negParticles = other_parameters_reco[i]

    lengthOfNeg = len(negParticles)
    lengthOfPos = len(posParticles)
    
    newPosArray = []
    newNegArray = []
    
    if lengthOfNeg==0 or lengthOfPos==0:
        continue;

    for l in range(len(posParticles)):
        reco_event_1_pos_parameters.append([[posParticles[l]]*(lengthOfNeg-l)][0])
    
    for j in range(len(posParticles)):
        newNegArray.append(negParticles[j:])
        
    a = flatten(newNegArray)
    reco_event_1_neg_parameters.append(a)

In [50]:
flatten_pos = [flatten(new_positive_mu[i]) for i in range(len(new_positive_mu))]

In [51]:
flatten_pos_pram = [flatten(pos_mu_prams[i]) for i in range(len(pos_mu_prams))]

In [52]:
flatten_pos_pram_tp = [flatten(pos_mu_prams_tp[i]) for i in range(len(pos_mu_prams_tp))]

In [53]:
flatten_pos_tp = [flatten(new_positive_mu_tp[i]) for i in range(len(new_positive_mu_tp))]

In [54]:
flatten_pos_reco = flatten(reco_event_1_pos)

In [55]:
flatten_neg_reco = flatten(reco_event_1_neg)

In [56]:
flatten_pos_reco_parameters = flatten(reco_event_1_pos_parameters)

In [57]:
len(flatten_pos_reco)

60804290

In [58]:
len(flatten_neg_reco)

60804290

In [59]:
len(reco_event_1_pos)

1039665

In [60]:
len(flatten_pos_reco_parameters)

60804290

In [61]:
flatten_neg_reco_parameters = flatten(reco_event_1_neg_parameters)

In [90]:
flatten_neg_reco[0]

{'rho': 2.4567391872406006,
 'phi': -0.43091803789138794,
 'eta': 0.21131443977355957,
 'tau': 0.1056583755}

In [193]:
# flatten_neg_reco_1 = flatten(flatten_neg_reco)

In [194]:
# len(flatten_neg_reco_1)

243217160

In [61]:
momentum_array = []
for i in range(len(new_negative_mu)):
    momentum_array.append(list(np.array(flatten_pos[i])+np.array(new_negative_mu[i])))

In [62]:
momentum_array_tp = []
for i in range(len(new_negative_mu_tp)):
    momentum_array_tp.append(list(np.array(flatten_pos_tp[i])+np.array(new_negative_mu_tp[i])))

In [63]:
# momentum_array_reco = []
# for i in range(len(flatten_pos_reco)):
#     momentum_array_reco.append(flatten_pos_reco[i]+flatten_neg_reco[i])

In [64]:
len(momentum_array_tp)

1554

In [65]:
momentum_flattened = flatten(momentum_array)

In [66]:
momentum_flattened_tp = flatten(momentum_array_tp)

In [96]:
len(z0_cut )

60804290

In [98]:
flatten_pos_reco_array = np.array(flatten_pos_reco)

In [101]:
flatten_neg_reco_array = np.array(flatten_neg_reco)

In [ ]:
delta_z0_reco = []
for i in range(len(flatten_neg_reco_parameters)):
    delta_z0_reco.append(flatten_neg_reco_parameters[i][0]-flatten_pos_reco_parameters[i][0])

In [ ]:
delta_z0_reco_array = np.array(delta_z0_reco)

In [ ]:
z0_cut_below = delta_z0_reco_array>-0.5
z0_cut_above = delta_z0_reco_array<0.5

z0_cut = np.logical_and(z0_cut_below,z0_cut_above)

In [99]:
reco_pos_z0_cut = flatten_pos_reco_array[z0_cut]

In [102]:
reco_neg_z0_cut = flatten_neg_reco_array[z0_cut]

In [103]:
len(reco_neg_z0_cut)

6426137

In [67]:
invariant_mass = []
for i in range(len(momentum_flattened)):
    invariant_mass.append(momentum_flattened[i].mass)

In [68]:
# invariant_mass_reco = []
# for i in range(len(momentum_array_reco)):
#     invariant_mass_reco.append(momentum_array_reco[i].mass)

In [69]:
invariant_mass_tp = []
for i in range(len(momentum_flattened_tp)):
    invariant_mass_tp.append(momentum_flattened_tp[i].mass)

In [70]:
pt_tp = []
for i in range(len(momentum_flattened_tp)):
    pt_tp.append(momentum_flattened_tp[i].pt)

In [71]:
pt = []
for i in range(len(momentum_flattened)):
    pt.append(momentum_flattened[i].pt)

In [72]:
# pt_reco = []
# for i in range(len(momentum_array_reco)):
#     pt_reco.append(momentum_array_reco[i].pt)

In [73]:
# len(pt_reco)

In [74]:
neg_prams_list = []
for i in range(len(neg_mu_prams)):
    sub_list = []
    for j in neg_mu_prams[i]:
        sub_list.append(j.tolist())
        
    neg_prams_list.append(sub_list)

In [75]:
neg_prams_list_tp = []
for i in range(len(neg_mu_prams_tp)):
    sub_list = []
    for j in neg_mu_prams_tp[i]:
        sub_list.append(j.tolist())
        
    neg_prams_list_tp.append(sub_list)

In [76]:
pos_prams_list = []
for i in range(len(flatten_pos_pram)):
    sub_list = []
    for j in flatten_pos_pram[i]:
        sub_list.append(j.tolist())
        
    pos_prams_list.append(sub_list)

In [77]:
pos_prams_list_tp = []
for i in range(len(flatten_pos_pram_tp)):
    sub_list = []
    for j in flatten_pos_pram_tp[i]:
        sub_list.append(j.tolist())
        
    pos_prams_list_tp.append(sub_list)

In [78]:
neg_prams_list[0]

[[-2.2265625, -999.0], [4.98046875, -999.0]]

In [80]:
all_z0_pos = []
for i in range(len(pos_prams_list)):
    event_array = []
    for j in range(len(pos_prams_list[i])):
        event_array.append(pos_prams_list[i][j][0])
    all_z0_pos.append(event_array)
    
all_z0_neg = []
for i in range(len(neg_prams_list)):
    event_array = []
    for j in range(len(neg_prams_list[i])):
        event_array.append(neg_prams_list[i][j][0])
    all_z0_neg.append(event_array)

In [81]:
all_z0_pos_tp = []
for i in range(len(pos_prams_list_tp)):
    event_array = []
    for j in range(len(pos_prams_list_tp[i])):
        event_array.append(pos_prams_list_tp[i][j][0])
    all_z0_pos_tp.append(event_array)
    
all_z0_neg_tp = []
for i in range(len(neg_prams_list_tp)):
    event_array = []
    for j in range(len(neg_prams_list_tp[i])):
        event_array.append(neg_prams_list_tp[i][j][0])
    all_z0_neg_tp.append(event_array)

In [82]:
len(invariant_mass)

2030

In [83]:
invariant_mass_array = np.array(invariant_mass)
negative_flatten = flatten(new_negative_mu)
positive_flatten = flatten(flatten_pos)

negative_array = np.array(negative_flatten)
positive_array = np.array(positive_flatten)


larger_background = invariant_mass_array>3.2
smaller_background = invariant_mass_array<3.0

lower_J_range = invariant_mass_array>3.0
upper_J_range = invariant_mass_array<3.2

In [84]:
J_range_truth = np.logical_and(lower_J_range,upper_J_range)
background_range_truth = np.logical_or(larger_background,smaller_background)

In [85]:
invariant_mass_signal = invariant_mass_array[J_range_truth]
negative_mu_signal = negative_array[J_range_truth]
positive_mu_signal = positive_array[J_range_truth]

In [86]:
negative_mu_background = negative_array[background_range_truth]
positive_mu_background = positive_array[background_range_truth]

In [87]:
eta_neg_signal = []
for i in range(len(negative_mu_signal)):
    eta_neg_signal.append(negative_mu_signal[i].eta)
        
phi_neg_signal = []
for i in range(len(negative_mu_signal)):
    phi_neg_signal.append(negative_mu_signal[i].phi)
        
pt_neg_signal = []
for i in range(len(negative_mu_signal)):
    pt_neg_signal.append(negative_mu_signal[i].pt)

In [88]:
eta_neg_background = []
for i in range(len(negative_mu_background)):
    eta_neg_background.append(negative_mu_background[i].eta)
        
phi_neg_background = []
for i in range(len(negative_mu_background)):
    phi_neg_background.append(negative_mu_background[i].phi)
        
pt_neg_background = []
for i in range(len(negative_mu_background)):
    pt_neg_background.append(negative_mu_background[i].pt)

In [89]:
eta_pos_signal = []
for i in range(len(positive_mu_signal)):
    eta_pos_signal.append(positive_mu_signal[i].eta)
        
phi_pos_signal = []
for i in range(len(positive_mu_signal)):
    phi_pos_signal.append(positive_mu_signal[i].phi)
        
pt_pos_signal = []
for i in range(len(positive_mu_signal)):
    pt_pos_signal.append(positive_mu_signal[i].pt)

In [90]:
eta_pos_background = []
for i in range(len(positive_mu_background)):
    eta_pos_background.append(positive_mu_background[i].eta)
        
phi_pos_background = []
for i in range(len(positive_mu_background)):
    phi_pos_background.append(positive_mu_background[i].phi)
        
pt_pos_background = []
for i in range(len(positive_mu_background)):
    pt_pos_background.append(positive_mu_background[i].pt)

In [105]:
reco_pos_z0_cut[0].eta

0.21131443977355957

In [106]:
eta_neg_reco = []
for i in range(len(reco_pos_z0_cut)):
    eta_neg_reco.append(reco_pos_z0_cut[i].eta)

In [109]:
eta_pos_reco = []
for i in range(len(reco_neg_z0_cut)):
    eta_pos_reco.append(reco_neg_z0_cut[i].eta)

In [110]:
len(eta_pos_reco)

6426137

In [113]:
len(np.array(eta_neg_reco)-1)

6426137

In [140]:
delta_eta_reco = np.abs(np.array(eta_pos_reco))-np.abs(np.array(eta_neg_reco))

In [126]:
delta_eta_reco_array = np.array(delta_eta_reco)

In [127]:
eta_cut_below = delta_eta_reco_array>-1
eta_cut_above = delta_eta_reco_array<1
eta_cut = np.logical_and(eta_cut_below,eta_cut_above)

In [131]:
reco_pos_z0_eta_cut = reco_pos_z0_cut[eta_cut]

In [132]:
reco_neg_z0_eta_cut = reco_neg_z0_cut[eta_cut]

In [133]:
len(reco_pos_z0_eta_cut)

4689451

In [134]:
len(reco_neg_z0_eta_cut)

4689451

In [135]:
phi_pos_reco = []
for i in range(len(reco_pos_z0_eta_cut)):
    phi_pos_reco.append(reco_pos_z0_eta_cut[i].phi)

In [136]:
phi_neg_reco = []
for i in range(len(reco_neg_z0_eta_cut)):
    phi_neg_reco.append(reco_neg_z0_eta_cut[i].phi)

In [141]:
delta_phi_reco = np.abs(np.array(phi_pos_reco))-np.abs(np.array(phi_neg_reco))

In [142]:
delta_phi_reco_array = np.array(delta_phi_reco)

In [143]:
len(delta_phi_reco)

4689451

In [144]:
phi_cut_below = delta_phi_reco_array>-1
phi_cut_above = delta_phi_reco_array<1

phi_cut = np.logical_and(phi_cut_below,phi_cut_above)

In [145]:
reco_pos_z0_eta_phi_cut = reco_pos_z0_eta_cut[phi_cut]

In [146]:
reco_neg_z0_eta_phi_cut = reco_neg_z0_eta_cut[phi_cut]

In [147]:
len(reco_pos_z0_eta_phi_cut)

3056836

In [148]:
len(reco_neg_z0_eta_phi_cut)

3056836

In [149]:
pt_neg_reco = []
for i in range(len(reco_neg_z0_eta_phi_cut)):
    pt_neg_reco.append(reco_neg_z0_eta_phi_cut[i].pt)

In [150]:
len(pt_neg_reco)

3056836

In [151]:
pt_pos_reco = []
for i in range(len(reco_pos_z0_eta_phi_cut)):
    pt_pos_reco.append(reco_pos_z0_eta_phi_cut[i].pt)

In [152]:
len(pt_pos_reco)

3056836

In [153]:
higher_pt_reco = []
lower_pt_reco = []
for i in range(len(pt_neg_reco)):
    if pt_neg_reco[i]>pt_pos_reco[i]:
        higher_pt_reco.append(pt_neg_reco[i])
        lower_pt_reco.append(pt_pos_reco[i])
    else:
        higher_pt_reco.append(pt_pos_reco[i])
        lower_pt_reco.append(pt_neg_reco[i])

In [154]:
higher_pt_reco_array = np.array(higher_pt_reco)

In [158]:
len(higher_pt_reco_array)

3056836

In [155]:
pt_cut = higher_pt_reco_array>3

In [159]:
len(pt_cut)

3056836

In [160]:
len(reco_neg_z0_eta_phi_cut)

3056836

In [161]:
reco_neg_z0_eta_phi_pt_cut = reco_neg_z0_eta_phi_cut[pt_cut]

In [163]:
len(reco_neg_z0_eta_phi_pt_cut)

1232317

In [162]:
reco_pos_z0_eta_phi_pt_cut = reco_pos_z0_eta_phi_cut[pt_cut]

In [172]:
reco_pos_z0_eta_phi_pt_cut[2]

<MomentumRecord4D {rho: 3.22, phi: -0.217, eta: ..., ...} type='Momentum4D[...'>

In [173]:
reco_neg_z0_eta_phi_pt_cut[2]+reco_pos_z0_eta_phi_pt_cut[2]

<MomentumRecord4D {rho: 5.1, phi: -0.608, eta: 0.399, ...} type='Momentum4D...'>

In [176]:
np.array(reco_neg_z0_eta_phi_pt_cut[0])+np.array(reco_pos_z0_eta_phi_pt_cut[0])

<MomentumRecord4D {rho: 6.44, phi: -0.217, eta: ..., ...} type='Momentum4D[...'>

In [ ]:
momentum_array_reco_new = np.array(reco_neg_z0_eta_phi_pt_cut)+np.array(reco_pos_z0_eta_phi_pt_cut)

In [174]:
momentum_array_reco = []
for i in range(len(reco_pos_z0_eta_phi_pt_cut)):
    momentum_array_reco.append(reco_pos_z0_eta_phi_pt_cut[i]+reco_neg_z0_eta_phi_pt_cut[i])

KeyboardInterrupt: 

In [ ]:
invariant_mass_reco = []
for i in range(len(momentum_array_reco)):
    invariant_mass_reco.append(momentum_array_reco[i].mass)

In [107]:
# eta_neg_reco = []
# for i in range(len(flatten_neg_reco)):
#     eta_neg_reco.append(flatten_neg_reco[i].eta)
        
# phi_neg_reco = []
# for i in range(len(flatten_neg_reco)):
#     phi_neg_reco.append(flatten_neg_reco[i].phi)
        
# pt_neg_reco = []
# for i in range(len(flatten_neg_reco)):
#     pt_neg_reco.append(flatten_neg_reco[i].pt)

In [ ]:
# eta_pos_reco = []
# for i in range(len(flatten_pos_reco)):
#     eta_pos_reco.append(flatten_pos_reco[i].eta)
        
# phi_pos_reco = []
# for i in range(len(flatten_pos_reco)):
#     phi_pos_reco.append(flatten_pos_reco[i].phi)
        
# pt_pos_reco = []
# for i in range(len(flatten_pos_reco)):
#     pt_pos_reco.append(flatten_pos_reco[i].pt)

In [ ]:
len(eta_pos_reco)

In [ ]:
# higher_pt_reco = []
# lower_pt_reco = []
# for i in range(len(pt_neg_reco)):
#     if pt_neg_reco[i]>pt_pos_reco[i]:
#         higher_pt_reco.append(pt_neg_reco[i])
#         lower_pt_reco.append(pt_pos_reco[i])
#     else:
#         higher_pt_reco.append(pt_pos_reco[i])
#         lower_pt_reco.append(pt_neg_reco[i])

In [ ]:
higher_pt_signal = []
lower_pt_signal = []
for i in range(len(pt_neg_signal)):
    if pt_neg_signal[i]>pt_pos_signal[i]:
        higher_pt_signal.append(pt_neg_signal[i])
        lower_pt_signal.append(pt_pos_signal[i])
    else:
        higher_pt_signal.append(pt_pos_signal[i])
        lower_pt_signal.append(pt_neg_signal[i])

In [ ]:
higher_pt_background = []
lower_pt_background = []
for i in range(len(pt_neg_background)):
    if pt_neg_background[i]>pt_pos_background[i]:
        higher_pt_background.append(pt_neg_background[i])
        lower_pt_background.append(pt_pos_background[i])
    else:
        higher_pt_background.append(pt_pos_background[i])
        lower_pt_background.append(pt_neg_background[i])

In [ ]:
delta_z0 = []
for i in range(len(all_z0_neg)):
    for j in range(len(all_z0_neg[i])):
        delta_z0.append(all_z0_pos[i][j]-all_z0_neg[i][j])

In [ ]:
delta_z0_tp = []
for i in range(len(all_z0_neg_tp)):
    for j in range(len(all_z0_neg_tp[i])):
        delta_z0_tp.append(all_z0_pos_tp[i][j]-all_z0_neg_tp[i][j])

In [ ]:
delta_z0_tp[0]

In [ ]:
flatten_pos_reco_parameters[0]

In [ ]:
delta_eta = []
for i in range(len(eta_neg_signal)):
    delta_eta.append(np.abs(eta_pos_signal[i])-np.abs(eta_neg_signal[i]))
    
delta_phi = []
for i in range(len(phi_neg_signal)):
    delta_phi.append(np.abs(phi_neg_signal[i])-np.abs(phi_pos_signal[i]))
    
delta_pt = []
for i in range(len(pt_neg_signal)):
    delta_pt.append(pt_neg_signal[i]-pt_pos_signal[i])

In [ ]:
# delta_eta_reco = []
# for i in range(len(eta_neg_reco)):
#     delta_eta_reco.append(np.abs(eta_pos_reco[i])-np.abs(eta_neg_reco[i]))

    
# delta_pt_reco = []
# for i in range(len(pt_neg_reco)):
#     delta_pt_reco.append(pt_neg_reco[i]-pt_pos_reco[i])

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(invariant_mass_tp,bins=200,range=(3.09,3.105),histtype='step',label = '2030 particles, 1554 events')
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('Invariant Mass (GeV)',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title(" Invariant Mass of tp",fontsize = 20)
plt.legend(loc='upper left',prop={'size': 15})

plt.show

In [ ]:
len(invariant_mass_tp)

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(invariant_mass_reco,bins=150,range=(0,10),histtype='step',label = '8911 particles')
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('Invariant Mass (GeV)',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title(" Invariant Mass of trk",fontsize = 20)
plt.legend(loc='upper left',prop={'size': 15})

plt.show

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(invariant_mass_signal,bins=100,range=(0,10),histtype='step',label = '856 particles, 1554 events')
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('Invariant Mass (GeV)',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title(" Invariant Mass of matchtrk",fontsize = 20)
plt.legend(loc='upper left',prop={'size': 15})

plt.show

In [ ]:
len(invariant_mass_signal)

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(delta_eta,bins=200,range=(-10,10),histtype='step',label = 'matchtrk', density = True)
plt.hist(delta_eta_reco,bins=200,range=(-10,10),histtype='step',label = 'trk', density = True)
# plt.hist(delta_eta_pt,bins=200,range=(-10,10),histtype='step',label = 'trk', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('Delta eta',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title(" Delta eta",fontsize = 20)
plt.legend(loc='upper left',prop={'size': 15})

plt.show

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(delta_phi,bins=200,range=(-10,10),histtype='step',label = 'matchtrk', density = True)
plt.hist(delta_phi_reco,bins=200,range=(-10,10),histtype='step',label = 'trk', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('delta phi',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title(" Delta Phi",fontsize = 20)
plt.legend(loc='upper left',prop={'size': 15})

plt.show

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(delta_pt,bins=200,range=(-10,10),histtype='step',label = 'matchtrk', density = True)
plt.hist(delta_pt_reco,bins=200,range=(-10,10),histtype='step',label = 'trk', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('Delta pt (GeV)',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title(" Delta pt",fontsize = 20)
plt.legend(loc='upper left',prop={'size': 15})

plt.show

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(higher_pt_signal,bins=100,range=(0,15),histtype='step',label = 'matchtrk higher pt signal tracks', density = True)
plt.hist(higher_pt_reco,bins=100,range=(0,15),histtype='step',label = 'trk higher pt tracks', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('pt (GeV)',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title("pt",fontsize = 20)
plt.legend(loc='upper right',prop={'size': 15})

plt.show

In [ ]:
len(higher_pt_reco_array[higher_pt_reco_array>3])

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(lower_pt_signal,bins=200,range=(0,10),histtype='step',label = 'matchtrk lower pt signal tracks', density = True)
plt.hist(lower_pt_reco,bins=200,range=(0,10),histtype='step',label = 'trk lower pt tracks', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('pt (GeV)',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title("pt",fontsize = 20)
plt.legend(loc='upper right',prop={'size': 15})

plt.show

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(pt_reco,bins=200,range=(0,10),histtype='step',label = 'matchtrk higher pt signal tracks', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('pt (GeV)',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title("pt",fontsize = 20)
plt.legend(loc='upper right',prop={'size': 15})

plt.show

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(higher_pt_signal,bins=200,range=(0,10),histtype='step',label = 'matchtrk higher pt signal tracks', density = True)
plt.hist(higher_pt_background,bins=200,range=(0,10),histtype='step',label = 'matchtrk higher pt background tracks', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('pt (GeV)',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title("pt",fontsize = 20)
plt.legend(loc='upper right',prop={'size': 15})

plt.show

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(lower_pt_signal,bins=200,range=(0,10),histtype='step',label = 'matchtrk lower pt signal tracks', density = True)
plt.hist(lower_pt_background,bins=200,range=(0,10),histtype='step',label = 'matchtrk lower pt background tracks', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('pt (GeV)',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title("pt",fontsize = 20)
plt.legend(loc='upper right',prop={'size': 15})

plt.show

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(delta_z0,bins=100,range=(-10,10),histtype='step',label = 'matchtrk',density = True)
plt.hist(delta_z0_reco,bins=100,range=(-10,10),histtype='step',label = 'trk', density = True)
# plt.hist(delta_z0_tp,bins=100,range=(-1,1),histtype='step',label = 'tp', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('delta z0',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title(" Delta z0",fontsize = 20)
plt.legend(loc='upper left',prop={'size': 15})

plt.show

In [ ]:
invariant_mass_reco_array = np.array(invariant_mass_reco)
# delta_pt_reco_array = np.array(delta_pt_reco)

In [ ]:
len(higher_pt_reco)

In [ ]:
# pt_reco_array = np.array(pt_reco)

In [ ]:
# len(phi_eta_z0_cut)

In [ ]:
# eta_z0_cut = eta_cut[z0_cut]

In [ ]:
# phi_z0_cut = phi_cut[z0_cut]
# phi_eta_z0_cut = phi_z0_cut[eta_z0_cut]

In [ ]:
# invariant_mass_reco_z0_cut = invariant_mass_reco_array[z0_cut]

In [ ]:
# pt_z0_cut = pt_cut[z0_cut]
# pt_eta_z0_cut = pt_z0_cut[eta_z0_cut]
# pt_phi_eta_z0_cut = pt_eta_z0_cut[phi_eta_z0_cut]

In [ ]:
# len(invariant_mass_reco_z0_cut)

In [ ]:
# delta_eta_reco_z0_cut = delta_eta_reco_array[z0_cut]

In [ ]:
# invariant_mass_reco_eta_z0_cut = invariant_mass_reco_z0_cut[eta_z0_cut]

In [ ]:
# invariant_mass_reco_phi_eta_z0_cut = invariant_mass_reco_eta_z0_cut[phi_eta_z0_cut]

In [ ]:
# invariant_mass_reco_pt_phi_eta_z0_cut = invariant_mass_reco_phi_eta_z0_cut[pt_phi_eta_z0_cut]

In [ ]:
# delta_phi_reco_z0_cut = delta_phi_reco_array[z0_cut]

In [ ]:
# delta_pt_reco_z0_cut = delta_pt_reco_array[z0_cut]

In [ ]:
# plt.figure(figsize = (10,5))
# plt.hist(invariant_mass_reco_pt_phi_eta_z0_cut,bins=100,range=(0,15),histtype='step',label = 'trk after z0 cut')
# plt.ylabel('tracks in bin',fontsize = 15)
# plt.xlabel('Invariant Mass (GeV)',fontsize = 15)
# plt.rc('xtick', labelsize=10) 
# plt.rc('ytick', labelsize=10)
# plt.title(" Invariant Mass of trk after z0 cut",fontsize = 20)
# plt.legend(loc='upper left',prop={'size': 15})

# plt.show

In [ ]:
# len(invariant_mass_reco_pt_phi_eta_z0_cut)

In [ ]:
# plt.figure(figsize = (10,5))
# plt.hist(delta_eta,bins=200,range=(-10,10),histtype='step',label = 'matchtrk', density = True)
# plt.hist(delta_eta_reco_z0_cut,bins=200,range=(-10,10),histtype='step',label = 'trk after cut', density = True)
# plt.ylabel('tracks in bin',fontsize = 15)
# plt.xlabel('Delta eta',fontsize = 15)
# plt.rc('xtick', labelsize=10) 
# plt.rc('ytick', labelsize=10)
# plt.title(" Delta eta after z0 cut",fontsize = 20)
# plt.legend(loc='upper left',prop={'size': 15})

# plt.show

In [ ]:
# plt.figure(figsize = (10,5))
# plt.hist(delta_phi,bins=200,range=(-10,10),histtype='step',label = 'matchtrk', density = True)
# plt.hist(delta_phi_reco_z0_cut,bins=200,range=(-10,10),histtype='step',label = 'trk after cut', density = True)
# plt.ylabel('tracks in bin',fontsize = 15)
# plt.xlabel('Delta phi',fontsize = 15)
# plt.rc('xtick', labelsize=10) 
# plt.rc('ytick', labelsize=10)
# plt.title(" Delta phi after z0 cut",fontsize = 20)
# plt.legend(loc='upper left',prop={'size': 15})

# plt.show

In [ ]:
# plt.figure(figsize = (10,5))
# plt.hist(delta_pt,bins=200,range=(-10,10),histtype='step',label = 'matchtrk', density = True)
# plt.hist(delta_pt_reco_z0_cut,bins=200,range=(-10,10),histtype='step',label = 'trk after cut', density = True)
# plt.ylabel('tracks in bin',fontsize = 15)
# plt.xlabel('Delta pt',fontsize = 15)
# plt.rc('xtick', labelsize=10) 
# plt.rc('ytick', labelsize=10)
# plt.title(" Delta pt after z0 cut",fontsize = 20)
# plt.legend(loc='upper left',prop={'size': 15})

# plt.show

In [ ]:
# a = flatten(delta_eta_reco_z0_cut)
# print(len(a))

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(delta_z0_tp,bins=100,range=(-1,1),histtype='step',label = 'tp')
# plt.hist(delta_z0_reco,bins=100,range=(-1,1),histtype='step',label = 'trk', density = True)
# plt.hist(delta_z0_tp,bins=100,range=(-1,1),histtype='step',label = 'tp', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('Delta z0',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title(" Delta z0 tp",fontsize = 20)
plt.legend(loc='upper left',prop={'size': 15})

plt.show

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(delta_z0_reco,bins=100,range=(-10,10),histtype='step',label = 'trk')
# plt.hist(delta_z0_reco,bins=100,range=(-1,1),histtype='step',label = 'trk', density = True)
# plt.hist(delta_z0_tp,bins=100,range=(-1,1),histtype='step',label = 'tp', density = True)
plt.ylabel('tracks in bin',fontsize = 15)
plt.xlabel('delta z0',fontsize = 15)
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.title("Delta z0 trk",fontsize = 20)
plt.legend(loc='upper left',prop={'size': 15})

plt.show

In [ ]:
r = np.array(invariant_mass_tp)

larger_background = r>3.10
smaller_background = r<3.09

lower_J_range = r>3.09
upper_J_range = r<3.10

In [ ]:
J_range_truth = np.logical_and(lower_J_range,lower_J_range)
background_range_truth = np.logical_or(larger_background,smaller_background)

In [ ]:
delta_z0_array[0]

In [ ]:
# delta_eta_array = np.array(delta_eta)
# delta_phi_array = np.array(delta_phi)
# delta_pt_array = np.array(delta_pt)
# delta_z0_array = np.array(delta_z0)

In [ ]:
# delta_eta_J = delta_eta_array[J_range_truth]
# delta_eta_background = delta_eta_array[background_range_truth]

# delta_phi_J = delta_phi_array[J_range_truth]
# delta_phi_background = delta_phi_array[background_range_truth]

# delta_pt_J = delta_pt_array[J_range_truth]
# delta_pt_background = delta_pt_array[background_range_truth]

# delta_z0_J = delta_z0_array[J_range_truth]
# delta_z0_background = delta_z0_array[background_range_truth]

In [ ]:
# delta_eta_J

In [ ]:
# len(delta_eta_background)